# K - Fold Cross Validation

- DataSet :- Titanic Data Set
- Model:- Decision Tree

In [30]:
# importing necessary libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split , KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [66]:
# loading Data

data = pd.read_csv(r'C:\Users\Tuhin\OneDrive\Desktop\BIA PROJECT\titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
df = data.copy()

In [33]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [35]:
# Checking information()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [36]:
# Duplictes
df[df.duplicated()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [37]:
# Dropping irrelevant coloumns : ['Passenger ID' , 'Name' , 'Ticket' , 'cabin']
df.drop(columns = ['PassengerId' , 'Name' , 'Ticket' , 'Cabin'] , axis = 1 , inplace = True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [38]:
# Checking and sumup null value

df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [39]:
# Imputting Null Value
#age Column replacing my mean age
df['Age'].fillna(round(df['Age'].mean(),2) , inplace = True)

#Embarked column replacing by mode
df['Embarked'].fillna(df['Embarked'].mode()[0],inplace = True)

In [40]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### Getting dummy values for text columns

In [41]:
# Non-Numeric Columns
non_numeric_columns = df.select_dtypes(include = object).columns.to_list()
non_numeric_columns

['Sex', 'Embarked']

In [42]:
#Getting dummy values
df = pd.get_dummies(df,columns = non_numeric_columns , dtype = int)
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


###  Setting Feature and Target

In [43]:
#Setting Feature

X = df.drop(columns = ['Survived'], axis = 1)
y = df['Survived']

# Split dataset
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42)

#Data Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.

In [44]:
# Model Building and Training
model = DecisionTreeClassifier(random_state = 42)


#Model Training
model.fit(X_train_scaled , y_train)



DecisionTreeClassifier(random_state=42)

In [45]:
# Prediction and Accuracy

#y_predict = model.predict(X_test_scaled)

y_predict_test = model.predict(X_test_scaled)
y_predict_train = model.predict(X_train_scaled)

#Accuracy

#accuracy_based_on_test_value = metrics.accuracy_score(y_test,y_predict) # prediction based on test value
accuracy_based_on_test_value = metrics.accuracy_score(y_test,y_predict_test)

accuracy_based_on_trained_value = metrics.accuracy_score(y_train,y_predict_train)

print(f'{accuracy_based_on_test_value : .2f}')
print(f'{accuracy_based_on_trained_value: .2f}')

 0.79
 0.98


### K-Fold Cross validation

In [46]:
# Initializing a  5-Fold cross Validation based on test data

In [47]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [48]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.

# Accuracy After Cross Validation
cv_score_based_on_test_data_k5 = cross_val_score(estimator = model, X = X_test_scaled , y = y_test, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [49]:
print(cv_score_based_on_test_data_k5)

[0.69444444 0.69444444 0.75       0.80555556 0.8       ]


In [50]:
#Average  Score 
avg_accuracy_based_on_test_data_k5 = cv_score_based_on_test_data_k5.mean()
print (f'{avg_accuracy_based_on_test_data_k5 : .2f}')

 0.75


In [51]:
# Initializing a  5-Fold cross Validation based on trained data

In [52]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [55]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.
# Accuracy After Cross Validation
cv_score_based_on_trained_data_k5 = cross_val_score(estimator = model, X = X_train_scaled , y = y_train, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [56]:
print(cv_score_based_on_trained_data_k5)

[0.76223776 0.76923077 0.71126761 0.76760563 0.73943662]


In [58]:
#Average  Score 
avg_accuracy_based_on_trained_data_k5 = cv_score_based_on_trained_data_k5.mean()
print (f'{avg_accuracy_based_on_trained_data_k5 : .2f}')

 0.75


In [59]:
# Initializing a  10-Fold cross Validation based on test data

In [60]:
k = 10
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [61]:
# Accuracy After Cross Validation

cv_score = cross_val_score(estimator = model, X = X_test_scaled , y = y_test, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [62]:
print(cv_score)

[0.72222222 0.83333333 0.77777778 0.66666667 0.66666667 0.88888889
 0.83333333 0.72222222 0.72222222 0.88235294]


In [63]:
#Average  Score 
avg_accuracy_based_on_test_data_k10 = cv_score.mean()
print (f'{avg_accuracy_based_on_test_data_k10 : .2f}')

 0.77


In [64]:
# Initializing a  10-Fold cross Validation based on train data

In [65]:
k = 10
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [179]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.

# Accuracy After Cross Validation
cv_score = cross_val_score(estimator = model, X = X_train_scaled , y = y_train, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [180]:
print(cv_score)

[0.86111111 0.76388889 0.77464789 0.73239437 0.69014085 0.77464789
 0.76056338 0.78873239 0.74647887 0.81690141]


In [181]:
avg_accuracy_based_on_trained_data_k10 = cv_score.mean()
print(f'{avg_accuracy_based_on_trained_data_k10:.2f}')

0.77


In [103]:
# Initializing a  15-Fold cross Validation based on test data

In [104]:
k = 15
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [105]:
# Accuracy After Cross Validation

cv_score = cross_val_score(estimator = model, X = X_test_scaled , y = y_test, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [106]:
print(cv_score)

[0.58333333 0.83333333 0.83333333 0.91666667 0.66666667 0.58333333
 0.5        0.83333333 0.91666667 0.75       0.83333333 0.83333333
 0.83333333 0.83333333 0.81818182]


In [183]:
#Average  Score 
avg_accuracy_based_on_test_data_k15 = cv_score.mean()
print (f'{avg_accuracy_based_on_test_data_k15 : .2f}')

 0.77


In [72]:
# Initializing a  15-Fold cross Validation based on trained data

In [73]:
k = 15
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [74]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.
# Accuracy After Cross Validation
cv_score = cross_val_score(estimator = model, X = X_train_scaled , y = y_train, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [75]:
print(cv_score)

[0.83333333 0.79166667 0.79166667 0.72916667 0.83333333 0.66666667
 0.77083333 0.72340426 0.76595745 0.80851064 0.78723404 0.76595745
 0.72340426 0.76595745 0.87234043]


In [76]:
#Average  Score 
avg_accuracy_based_on_trained_data_k15 = cv_score.mean()
print (f'{avg_accuracy_based_on_trained_data_k15 : .2f}')

 0.78


In [108]:
# Initializing a  20-Fold cross Validation based on test data

In [109]:
k = 20
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [110]:
# Accuracy After Cross Validation

cv_score = cross_val_score(estimator = model, X = X_test_scaled , y = y_test, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [111]:
print(cv_score)

[0.66666667 0.88888889 0.77777778 0.77777778 1.         0.55555556
 0.44444444 0.66666667 0.55555556 0.77777778 0.88888889 0.88888889
 0.77777778 0.88888889 0.77777778 0.66666667 0.77777778 0.77777778
 1.         0.75      ]


In [112]:
#Average  Score 
avg_accuracy = cv_score.mean()
print (f'{avg_accuracy : .2f}')

 0.77


In [77]:
# Initializing a  20-Fold cross Validation based on train data

In [78]:
k = 20
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [ ]:
# Accuracy After Cross Validation

cv_score = cross_val_score(estimator = model,  X = X_train_scaled , y = y_train, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [79]:
print(cv_score)

[0.83333333 0.79166667 0.79166667 0.72916667 0.83333333 0.66666667
 0.77083333 0.72340426 0.76595745 0.80851064 0.78723404 0.76595745
 0.72340426 0.76595745 0.87234043]


In [80]:
#Average  Score 
avg_accuracy = cv_score.mean()
print (f'{avg_accuracy : .2f}')

 0.78


 - Model:-  Random Forest
 - Data Set :- Titanic Data Set

In [1]:
# importing necessary libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split , KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [2]:
# loading Data

data = pd.read_csv(r'C:\Users\Tuhin\OneDrive\Desktop\BIA PROJECT\titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df = data.copy()

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
# Duplictes
df[df.duplicated()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [8]:
# Dropping irrelevant coloumns : ['Passenger ID' , 'Name' , 'Ticket' , 'cabin']
df.drop(columns = ['PassengerId' , 'Name' , 'Ticket' , 'Cabin'] , axis = 1 , inplace = True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [9]:
# Checking and sumup null value

df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [10]:
# Imputting Null Value
#age Column replacing my mean age
df['Age'].fillna(round(df['Age'].mean(),2) , inplace = True)

#Embarked column replacing by mode
df['Embarked'].fillna(df['Embarked'].mode()[0],inplace = True)

In [11]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [12]:
# Non-Numeric Columns
non_numeric_columns = df.select_dtypes(include = object).columns.to_list()
non_numeric_columns

['Sex', 'Embarked']

In [13]:
#Getting dummy values
df = pd.get_dummies(df,columns = non_numeric_columns , dtype = int)
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [14]:
#Setting Feature

X = df.drop(columns = ['Survived'], axis = 1)
y = df['Survived']

# Split dataset
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42)

#Data Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.

In [15]:
# Model Building and Training
model = RandomForestClassifier(random_state = 42)


#Model Training
model.fit(X_train_scaled , y_train)



RandomForestClassifier(random_state=42)

In [16]:
# Prediction and Accuracy

#y_predict = model.predict(X_test_scaled)

y_predict_test = model.predict(X_test_scaled)
y_predict_train = model.predict(X_train_scaled)

#Accuracy

#accuracy_based_on_test_value = metrics.accuracy_score(y_test,y_predict) # prediction based on test value
accuracy_based_on_test_value = metrics.accuracy_score(y_test,y_predict_test)

accuracy_based_on_trained_value = metrics.accuracy_score(y_train,y_predict_train)

print(f'{accuracy_based_on_test_value : .2f}')
print(f'{accuracy_based_on_trained_value: .2f}')

 0.82
 0.98


### K-Fold Cross validation

In [ ]:
k = 5

In [17]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [18]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.

# Accuracy After Cross Validation
cv_score_based_on_test_data_k5 = cross_val_score(estimator = model, X = X_test_scaled , y = y_test, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [19]:
print(cv_score_based_on_test_data_k5)

[0.75       0.63888889 0.86111111 0.80555556 0.85714286]


In [20]:
#Average  Score 
avg_accuracy_based_on_test_data_k5 = cv_score_based_on_test_data_k5.mean()
print (f'{avg_accuracy_based_on_test_data_k5 : .2f}')

 0.78


In [ ]:
# Initializing a  5-Fold cross Validation based on trained data

In [21]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [22]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.
# Accuracy After Cross Validation
cv_score_based_on_trained_data_k5 = cross_val_score(estimator = model, X = X_train_scaled , y = y_train, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [23]:
print(cv_score_based_on_trained_data_k5)

[0.8041958  0.76223776 0.76760563 0.78873239 0.78873239]


In [24]:
avg_accuracy_based_on_trained_data_k5 = cv_score_based_on_trained_data_k5.mean()
print (f'{avg_accuracy_based_on_trained_data_k5 : .2f}')

 0.78


In [25]:
# Initializing a  5-Fold cross Validation based on full data

In [27]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [28]:
X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.
# Accuracy After Cross Validation
cv_score_based_on_full_data_k5 = cross_val_score(estimator = model, X = X , y = y, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [33]:
print(cv_score_based_on_full_data_k5)

[0.82122905 0.80337079 0.84269663 0.80337079 0.8258427 ]


In [34]:
avg_accuracy_based_on_full_data_k5 = cv_score_based_on_full_data_k5.mean()
print (f'{avg_accuracy_based_on_full_data_k5 : .2f}')

 0.82


### Gradiant boost

In [2]:
# importing necessary libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split , KFold , cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [3]:
# loading Data

data = pd.read_csv(r'C:\Users\Tuhin\OneDrive\Desktop\BIA PROJECT\titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df = data.copy()

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [8]:
# Duplictes
df[df.duplicated()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [9]:
# Dropping irrelevant coloumns : ['Passenger ID' , 'Name' , 'Ticket' , 'cabin']
df.drop(columns = ['PassengerId' , 'Name' , 'Ticket' , 'Cabin'] , axis = 1 , inplace = True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [10]:
# Checking and sumup null value

df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [11]:
# Imputting Null Value
#age Column replacing my mean age
df['Age'].fillna(round(df['Age'].mean(),2) , inplace = True)

#Embarked column replacing by mode
df['Embarked'].fillna(df['Embarked'].mode()[0],inplace = True)

In [12]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [13]:
# Non-Numeric Columns
non_numeric_columns = df.select_dtypes(include = object).columns.to_list()
non_numeric_columns

['Sex', 'Embarked']

In [14]:
#Getting dummy values
df = pd.get_dummies(df,columns = non_numeric_columns , dtype = int)
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [15]:
#Setting Feature

X = df.drop(columns = ['Survived'], axis = 1)
y = df['Survived']

# Split dataset
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42)

#Data Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

# Model Building and Training
model = GradientBoostingClassifier(random_state=42)

# Model Training
model.fit(X_train_scaled, y_train)

GradientBoostingClassifier(random_state=42)

In [18]:
# Prediction and Accuracy

#y_predict = model.predict(X_test_scaled)

y_predict_test = model.predict(X_test_scaled)
y_predict_train = model.predict(X_train_scaled)

#Accuracy

#accuracy_based_on_test_value = metrics.accuracy_score(y_test,y_predict) # prediction based on test value
accuracy_based_on_test_value = metrics.accuracy_score(y_test,y_predict_test)

accuracy_based_on_trained_value = metrics.accuracy_score(y_train,y_predict_train)

print(f'{accuracy_based_on_test_value : .2f}')
print(f'{accuracy_based_on_trained_value: .2f}')

 0.82
 0.91


## K - Fold Cross Validation

In [19]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [20]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.

# Accuracy After Cross Validation
cv_score_based_on_test_data_k5 = cross_val_score(estimator = model, X = X_test_scaled , y = y_test, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [22]:
print(cv_score_based_on_test_data_k5)

[0.75       0.66666667 0.86111111 0.77777778 0.82857143]


In [23]:
#Average  Score 
avg_accuracy_based_on_test_data_k5 = cv_score_based_on_test_data_k5.mean()
print (f'{avg_accuracy_based_on_test_data_k5 : .2f}')

 0.78


In [24]:
# Initializing a  5-Fold cross Validation based on trained data

In [25]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [26]:
#X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.
# Accuracy After Cross Validation
cv_score_based_on_trained_data_k5 = cross_val_score(estimator = model, X = X_train_scaled , y = y_train, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [27]:
print(cv_score_based_on_trained_data_k5)

[0.82517483 0.8041958  0.78873239 0.82394366 0.85915493]


In [28]:
avg_accuracy_based_on_trained_data_k5 = cv_score_based_on_trained_data_k5.mean()
print (f'{avg_accuracy_based_on_trained_data_k5 : .2f}')

 0.82


In [29]:
# Initializing a  5-Fold cross Validation based on full data

In [30]:
k = 5
kf = KFold(n_splits = k , shuffle = True , random_state = 42)

In [31]:
X_train_scaled = scaler.fit_transform(X_train) # fit_transform uses statics finding the mean and satandarddeviation of the data
#X_test_scaled = scaler.transform(X_test) # transform  data using previously found statics.
# Accuracy After Cross Validation
cv_score_based_on_full_data_k5 = cross_val_score(estimator = model, X = X , y = y, cv = kf ,scoring = 'accuracy' , n_jobs = -1)

In [32]:
print(cv_score_based_on_full_data_k5)


[0.81564246 0.81460674 0.85955056 0.80898876 0.83146067]


In [33]:
avg_accuracy_based_on_full_data_k5 = cv_score_based_on_full_data_k5.mean()
print (f'{avg_accuracy_based_on_full_data_k5 : .2f}')

 0.83
